In [1]:
from typing import List, Optional, Generator
import typing
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq
import operator

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Helper for N Ary Trees

In [4]:
class NAryNode:
    def __init__(self, val: int) -> None:
        self.val = val
        self.children: list['NAryNode'] = []

    @staticmethod
    def from_list(nums: list[int|None]) -> 'NAryNode|None':
        nodes: list[NAryNode] = []
        idx = -1
        for val in nums:
            if val is None:
                idx += 1
            else:
                curr = NAryNode(val)
                if nodes:
                    assert idx != -1, "Invalid input provided"
                    nodes[idx].children.append(curr)
                nodes.append(curr)

        return nodes[0] if nodes else None

    def to_list(self) -> list[int|None]:
        result: list[int|None] = [self.val, None]
        queue: collections.deque['NAryNode'] = collections.deque([self])
        while queue:
            curr = queue.popleft()
            for child in curr.children:
                queue.append(child)
                result.append(child.val)
            result.append(None)

        # Strip out all `None` from the end
        while result and result[-1] is None:
            result.pop()

        return result

# Testing the function
temp = NAryNode.from_list([1,None,3,2,4,None,5,6])
assert temp and temp.to_list() == [1,None,3,2,4,None,5,6]

CF Div 2 - 15th Aug 2024

In [5]:
def closest_point(N: int, points: list[int]) -> bool:
    return N == 2 and abs(points[0] - points[1]) > 1

# Testing the solution
assert closest_point(2, [3,8]) == True
assert closest_point(2, [5,6]) == False
assert closest_point(6, [1,2,3,4,5,10]) == False

Upsolvng CF Contest - Div 2

In [6]:
def game_with_doors(AL: int, AR: int, BL: int, BR: int) -> int:
    """
    For the overlapping portion, all doors must be closed
    For non overlapping portion, suffices to close just one door at each side
    """
    # Check for overlap
    oL, oR = max(AL, BL), min(AR, BR)
    if oL <= oR:
        locks = oR - oL
        if abs(AL - BL) > 0:
            locks += 1
        if abs(AR  - BR) > 0:
            locks += 1
        return locks
    else:
        return 1

# Testing the solution
assert game_with_doors(1, 2, 3, 4) == 1
assert game_with_doors(2, 5, 2, 5) == 3
assert game_with_doors(3, 7, 6, 7) == 2
assert game_with_doors(4, 5, 2, 8) == 3

In [7]:
def splitting_items(N: int, K: int, costs: list[int]) -> int:
    """
    * Playing optimally implies that
        - Alice tries to pick up the largest costing item available
        - Bob picks up the largest costing item available
        - Alice picks up items at odd indices, Bob at even indices (post sorting)
    * To minimize `A - B`, Bob should increase cost of items at even indices
    * We can update delta on the fly until K > 0
    """
    costs.sort()
    result = 0
    while costs:
        if len(costs) >= 2:
            delta = costs.pop() - costs.pop()
            if K < delta:
                result += delta - K
            K = max(0, K - delta)
        else:
            result += costs.pop()

    return result

# Testing the solution
assert splitting_items(3, 0, [10, 15, 12]) == 13
assert splitting_items(2, 5, [1, 10]) == 4
assert splitting_items(4, 6, [3, 1, 2, 4]) == 0
assert splitting_items(2, 4, [6, 9]) == 0
assert splitting_items(3, 1, [1, 1, 1]) == 1

In [8]:
def colored_portals(N: int, Q: int, cities: list[str], queries: list[tuple[int, int]]) -> list[int]:
    """
    If cities of i, j share a common color, shortest path to reach would abs(i - j)
    Otherwise, we check for common colors between target:
        - If no common colors return -1
        - Elif if there exists a common point between i, j return abs(i - j)
        - Else search for points to left of i and right i with common portal and check both distances, return least of two (use binary search)
    """

    # Store to hashmap for quick lookup times
    portal_groups: collections.defaultdict[str, list] = collections.defaultdict(list)
    for i, portal_type in enumerate(cities):
        portal_groups[portal_type].append(i)

    # Process the queries
    results: list[int] = []
    for i, j in queries:
        i, j = i - 1, j - 1
        a, b, c, d = cities[i][0], cities[i][1], cities[j][0], cities[j][1]

        # All 4 distinct portals
        if len(set([a, b, c, d])) == 4:
            distance = math.inf
            for comb in [(a, c), (a, d), (b, c), (b, d)]:
                common = "".join(sorted(comb))
                idx = bisect.bisect(portal_groups[common], i)
                left, right = idx - 1, idx
                if 0 <= left < len(portal_groups[common]):
                    distance = min(distance, abs(portal_groups[common][left] - i) + abs(portal_groups[common][left] - j))

                if 0 <= right < len(portal_groups[common]):
                    distance = min(distance, abs(portal_groups[common][right] - i) + abs(portal_groups[common][right] - j))

            results.append(int(distance) if not math.isinf(distance) else -1)

        # Atleast one color in common
        else:
           results.append(abs(j - i))

    return results

# Testing the solution
assert colored_portals(6, 6, ["RY", "RY", "GY", "RY", "GB", "RY"], [(2,5), (4,5), (1,4), (4,1), (5,4), (5,2)]) == [3,3,3,3,3,3]
assert colored_portals(9, 3, ["RY", "RY", "BR", "BR", "BR", "BR", "GY", "GY", "RY"], [(3,7), (2,5), (6,7)]) == [6,3,5]
assert colored_portals(4, 5, ["BR","BR","GY","GR"], [(1,2), (3,1), (4,4), (1,4), (4,2)]) == [1,4,0,3,2]

Find MEX of a set

In [9]:
def MEX(N: int, nums: list[int]) -> int:
    """
    There is a more optimized version for cases where nums are frequently updated.
    """
    xor = functools.reduce(operator.xor, nums, 0)
    missing = functools.reduce(operator.xor, range(N), xor)
    return missing

# Testing the solution
assert MEX(5, [0,2,3,4]) == 1
assert MEX(3, [1,2]) == 0
assert MEX(3, [2,0]) == 1
assert MEX(4, [2,0,1]) == 3

In [10]:
# Leetcode Weekly: 18th Aug 2024

In [11]:
def countKConstraintSubstrings(S: str, K: int) -> int:
    N = len(S)
    total = 0
    for i in range(N):
        zeros = ones = 0
        for j in range(i, N):
            if S[j] == '1':
                ones += 1
            else:
                zeros += 1
            if ones <= K or zeros <= K:
                total += 1
            else:
                break
    return total

# Testing the solution
assert countKConstraintSubstrings("10101", 1) == 12
assert countKConstraintSubstrings("1010101", 2) == 25
assert countKConstraintSubstrings("11111", 1) == 15

In [12]:
def maxEnergyBoost(energyDrinkA: list[int], energyDrinkB: list[int]) -> int:
    @functools.cache
    def backtrack(i: int, prev: int = 0) -> int:
        if i == N:
            return 0
        else:
            max_energy = backtrack(i + 1, 0)
            if prev != 2:
                max_energy = max(max_energy, energyDrinkA[i] + backtrack(i + 1, 1))
            if prev != 1:
                max_energy = max(max_energy, energyDrinkB[i] + backtrack(i + 1, 2))
            return max_energy

    N = len(energyDrinkA)
    return backtrack(0)

# Testing the solution
assert maxEnergyBoost([1,3,1], [3,1,1]) == 5
assert maxEnergyBoost([4,1,1], [1,1,3]) == 7

CP Practice - 19th Aug 2024

In [13]:
def learning_languages(N: int, M: int, languages_known: list[tuple[int, list[int]]]) -> int:
    # Store the speakers who speak a particular language
    language_speakers: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for employee, (count, languages) in enumerate(languages_known):
        for language in languages:
            language_speakers[language].append(employee)

    # Perform a DFS traversal, we need to count the number of distinct groups
    visited: set[int] = set()
    know_nothings, groups = 0, 0
    for i in range(N):
        if i not in visited:
            if languages_known[i][0]:
                groups += 1
                stack: list[int] = [i]
                visited.add(i)
                while stack:
                    emp = stack.pop()
                    for language in languages_known[emp][1]:
                        for next_emp in language_speakers[language]:
                            if next_emp not in visited:
                                stack.append(next_emp)
                                visited.add(next_emp)
            else:
                know_nothings += 1

    return max(groups - 1, 0) + know_nothings

# Testing the solution
assert learning_languages(8, 7, [(0, []), (3, [1,2,3]), (1, [1]), (2, [5,4]), (2, [6,7]), (1, [3]), (2, [7,4]), (1, [1])]) == 2
assert learning_languages(2, 2, [(1, [2]), (0, [])]) == 1
assert learning_languages(5, 5, [(1, [2]), (2, [2,3]), (2, [3,4]), (2, [4,5]), (1, [5])]) == 0
assert learning_languages(2, 2, [(0, []), (0, [])]) == 2
assert learning_languages(3, 2, [(0, []), (1, [1]), (1, [2])]) == 2

CP Practice - 20th Aug 2024

In [14]:
def countSeniors(details: list[str]) -> int:
    return sum(map(lambda x: int(x[11:13]) > 60, details))

# Testing the solution
assert countSeniors(["7868190130M7522","5303914400F9211","9273338290F4010"]) == 2
assert countSeniors(["1313579440F2036","2921522980M5644"]) == 0

In [15]:
def canBeEqual(target: list[int], arr: list[int]) -> bool:
    return collections.Counter(target) == collections.Counter(arr)

# Testing the solution
assert canBeEqual([1,2,3,4], [2,4,1,3]) == True
assert canBeEqual([7], [7]) == True
assert canBeEqual([3,7,9], [3,7,11]) == False

CP Practice: 21th Aug 2024

In [16]:
def bertown_subway(N: int, stations: list[int]) -> int:
    # Compute the number of disjoint stations
    groups: collections.defaultdict[int, set[int]] = collections.defaultdict(set)
    color: int = 0
    visited: set[int] = set()
    for i in range(N):
        if i not in visited:
            curr, color = i, color + 1
            while curr not in visited:
                visited.add(curr)
                groups[color].add(curr)
                curr = stations[curr] - 1

    # We can combine any two groups - to maximize, join the last two groups
    group_sizes: list[int] = sorted(map(len, groups.values()))
    return sum(map(lambda x: x ** 2, group_sizes[:-2])) + sum(group_sizes[-2:]) ** 2

# Testing the solution
assert bertown_subway(5, [1,5,4,3,2]) == 17
assert bertown_subway(3, [2,1,3]) == 9

In [17]:
def findComplement(num: int) -> int:
    result, power = 0, 1
    while num:
        result += power if not num & 1 else 0
        num, power = num >> 1, power << 1
    return result

# Testing the solution
assert findComplement(5) == 2
assert findComplement(1) == 0

CP Practice: 24th Aug 2024

In [18]:
def k_complete_word(N: int, K: int, word: str) -> int:
    updates = 0
    visited: set[int] = set()
    for i in range(N):
        if i not in visited:
            chars: list[int] = [0] * 26
            max_, count = 0, 0
            for j in range(i, N, K):
                if j not in visited:
                    visited.add(j)
                    chars[ord(word[j]) - ord('a')] += 1
                    max_, count = max(max_, chars[ord(word[j]) - ord('a')]), count + 1
                if N - j - 1 not in visited:
                    visited.add(N - j - 1)
                    chars[ord(word[N - j - 1]) - ord('a')] += 1
                    max_, count = max(max_, chars[ord(word[N - j - 1]) - ord('a')]), count + 1
            updates += (count - max_)

    return updates

# Testing the solution
assert k_complete_word(36, 9, "hippopotomonstrosesquippedaliophobia") == 23
assert k_complete_word(21, 7, "wudixiaoxingxingheclp") == 16
assert k_complete_word(6, 2, "abaaba") == 2
assert k_complete_word(6, 3, "abaaba") == 0

CP Practice: 25th Aug 2024

In [19]:
def powerful_array(N: int, T: int, nums: list[int], queries: list[tuple[int, int]]) -> list[int]:
    """
    This requires implementation of Moo's algorithm.
    Divide the nums into blocks of size sqrt(N) each, we answer the queries in a special order.
    """
    class StepWiseResult:
        def __init__(self) -> None:
            # Data structure to maintain the `current` result
            self.freq: collections.defaultdict[int, int] = collections.defaultdict(int)
            self.result = 0
            self.curr_l = 0
            self.curr_r = -1

        def add(self, idx) -> None:
            count = self.freq[nums[idx]]
            self.result += ((count + 1) * (count + 1) * nums[idx]) - (count * count * nums[idx])
            self.freq[nums[idx]] += 1

        def remove(self, idx) -> None:
            count = self.freq[nums[idx]]
            self.result += ((count - 1) * (count - 1) * nums[idx]) - (count * count * nums[idx])
            self.freq[nums[idx]] -= 1

    # Block size of Sqrt(N)
    BLOCK_SIZE = int(math.sqrt(N))

    # Sort the left index by block number and right index in asc order if block num is odd else in desc order
    sorted_queries = sorted(enumerate(queries), key=lambda x: (x[1][0] // BLOCK_SIZE, x[1][1] if (x[1][0] // BLOCK_SIZE) % 2 == 0 else -x[1][1]))

    # Answer queries in a particular order
    results: list[int] = [-1] * T
    swr = StepWiseResult()
    curr_l, curr_r = 0, -1
    for idx, (L, R) in sorted_queries:
        L, R = L - 1, R - 1
        while curr_l > L:
            curr_l -= 1
            swr.add(curr_l)
        while curr_r < R:
            curr_r += 1
            swr.add(curr_r)
        while curr_l < L:
            swr.remove(curr_l)
            curr_l += 1
        while curr_r > R:
            swr.remove(curr_r)
            curr_r -= 1

        results[idx] = swr.result

    return results

# Testing the solution
assert powerful_array(8, 3, [1,1,2,2,1,3,1,1], [(2,7), (1,6), (2,7)]) == [20, 20, 20]
assert powerful_array(3, 2, [1,2,1], [(1,2), (1,3)]) == [3, 6]

LC Weekly Contest - 25th Aug 2024

In [20]:
def getFinalState(nums: list[int], K: int, multiplier: int) -> list[int]:
    heap: list[tuple[int, int]] = list(map(lambda x: (x[1], x[0]), enumerate(nums)))
    heapq.heapify(heap)
    for i in range(K):
        curr, idx = heapq.heappop(heap)
        heapq.heappush(heap, (curr * multiplier, idx))

    result: list[int] = [-1] * len(heap)
    for curr, idx in heap:
        result[idx] = curr % (10 ** 9 + 7)

    return result

# Testing the solution
assert getFinalState([2,1,3,5,6], 5, 2) == [8,4,6,5,6]
assert getFinalState([1,2], 3, 4) == [16, 8]
assert getFinalState([1,2], 3, 4) == [16, 8]
assert getFinalState([100000,2000], 2, 1000000) == [999999307,999999993]

In [21]:
def countPairs(nums: list[int]) -> int:
    def check_almost_equal(n1: int, n2: int) -> bool:
        n1_str, n2_str = str(n1), str(n2)
        max_width = max(len(n1_str), len(n2_str))
        n1_str, n2_str = n1_str.zfill(max_width), n2_str.zfill(max_width)
        f1, f2 = collections.Counter(n1_str), collections.Counter(n2_str)
        non_match_pos = 0
        for ch1, ch2 in zip(n1_str, n2_str):
            if ch1 != ch2:
                non_match_pos += 1

        return f1 == f2 and non_match_pos <= 2

    N = len(nums)
    count = 0
    for i in range(N):
        for j in range(i + 1, N):
            count += int(check_almost_equal(nums[i], nums[j]))

    return count

# Testing the solution
assert countPairs([3,12,30,17,21]) == 2
assert countPairs([1,1,1,1,1]) == 10
assert countPairs([123,231]) == 0

Codeforces Div - 2: 25th Aug 2024

In [22]:
def turtle_and_good_strings(N: int, S: str) -> bool:
    return S[0] != S[-1]

# Testing the solution
assert turtle_and_good_strings(2, "aa") == False
assert turtle_and_good_strings(3, "aba") == False
assert turtle_and_good_strings(4, "abcb") == True
assert turtle_and_good_strings(12, "abcabcabcabc") == True

In [23]:
def turtle_and_piggy_are_playing_a_game(N: int, nums: list[int]) -> int:
    nums.sort()
    return nums[N // 2]

# Testing the solution
assert turtle_and_piggy_are_playing_a_game(2, [1,2]) == 2
assert turtle_and_piggy_are_playing_a_game(3, [1,1,2]) == 1
assert turtle_and_piggy_are_playing_a_game(3, [1,2,3]) == 2
assert turtle_and_piggy_are_playing_a_game(5, [3,1,2,2,3]) == 2
assert turtle_and_piggy_are_playing_a_game(10, [10,2,5,2,7,9,2,5,10,7]) == 7

In [24]:
def turtle_and_good_pairs(N: int, S: str) -> str:
    freq: list[int] = [0] * 26
    for ch in S:
        freq[ord(ch) - ord('a')] += 1

    result: list[str] = []
    i = 0
    while N:
        if freq[i] > 0:
            result.append(chr(i + ord('a')))
            N -= 1
            freq[i] -= 1
        i = (i + 1) % 26

    return "".join(result)

# Testing the solution
def test_turtle_and_good_pairs(S: str, ans: str) -> None:
    def count(s: str) -> int:
        N = len(s)
        count = 0
        for i in range(N):
            chars: set[str] = {s[i]}
            for j in range(i + 1, N):
                chars.add(s[j])
                if s[i] == s[j] or len(chars) > 2:
                    count += 1
        return count

    assert count(S) == count(ans)

test_turtle_and_good_pairs(turtle_and_good_pairs(3, "abc"), "acb")
test_turtle_and_good_pairs(turtle_and_good_pairs(5, "edddf"), "ddedf")
test_turtle_and_good_pairs(turtle_and_good_pairs(10, "codeforces"), "codeforces")
test_turtle_and_good_pairs(turtle_and_good_pairs(6, "turtle"), "urtlet")

DSA Touch: 26th Aug 2024

In [25]:
def postorder_N_ary_opt1(root: 'NAryNode|None') -> list[int]:
    result: list[int] = []
    stack: list[tuple['NAryNode|None', int]] = [(root, 0)]
    while stack:
        curr, idx = stack[-1]
        if not curr or idx == len(curr.children):
            if curr:
                result.append(curr.val)
            stack.pop()
        else:
            stack[-1] = curr, idx + 1
            stack.append((curr.children[idx], 0))

    return result

# Testing the solution
assert postorder_N_ary_opt1(NAryNode.from_list([1,None,3,2,4,None,5,6])) == [5,6,3,2,4,1]
assert postorder_N_ary_opt1(NAryNode.from_list([1,None,2,3,4,5,None,None,6,7,None,8,None,9,10,None,None,11,None,12,None,13,None,None,14])) == [2,6,14,11,7,3,12,8,4,13,9,10,5,1]

def postorder_N_ary_opt2(root: 'NAryNode|None') -> list[int]:
    result: list[int] = []
    stack: list[tuple['NAryNode|None', bool]] = [(root, False)]
    while stack:
        curr, completed = stack.pop()
        if not curr:
            continue
        elif completed:
            result.append(curr.val)
        else:
            stack.append((curr, True))
            for children in curr.children[::-1]:
                stack.append((children, False))

    return result

# Testing the solution
assert postorder_N_ary_opt2(NAryNode.from_list([1,None,3,2,4,None,5,6])) == [5,6,3,2,4,1]
assert postorder_N_ary_opt2(NAryNode.from_list([1,None,2,3,4,5,None,None,6,7,None,8,None,9,10,None,None,11,None,12,None,13,None,None,14])) == [2,6,14,11,7,3,12,8,4,13,9,10,5,1]

In [26]:
def construct2DArray(original: list[int], M: int, N: int) -> list[list[int]]:
    if len(original) != M * N:
        return []
    else:
        results: list[list[int]] = [[]]
        for i in range(len(original)):
            results[-1].append(original[i])
            if (i + 1) % N == 0:
                results.append([])

    results.pop()
    return results

# Testing the solution
assert construct2DArray([1,2,3,4], 2, 2) == [[1,2], [3,4]]
assert construct2DArray([1,2,3], 1, 3) == [[1,2,3]]
assert construct2DArray([1,2], 1, 1) == []

LC Weekly: 1st Sept 2024

In [27]:
def checkTwoChessboards(coord1: str, coord2: str) -> bool:
    c1, c2 = (ord(coord1[0]) - ord('a'), int(coord1[1]) - 1), (ord(coord2[0]) - ord('a'), int(coord2[1]) - 1)
    checkColor = lambda x: x[0] % 2 == x[1] % 2
    return checkColor(c1) == checkColor(c2)

# Testing the solution
assert checkTwoChessboards("a1", "c3") == True
assert checkTwoChessboards("a1", "h3") == False
assert checkTwoChessboards("b4", "h6") == True

In [28]:
def resultsArray(queries: list[list[int]], K: int) -> list[int]:
    def distance(pt: list[int]) -> int:
        return abs(pt[0]) + abs(pt[1])

    results: list[int] = []
    heap: list[int] = []
    for pt in queries:
        heapq.heappush(heap, -distance(pt))
        if len(heap) > K:
            heapq.heappop(heap)
        results.append(-heap[0] if len(heap) == K else -1)
    return results

# Testing the solution
assert resultsArray([[1,2],[3,4],[2,3],[-3,0]], 2) == [-1, 7, 5, 3]
assert resultsArray([[5,5],[4,4],[3,3]], 1) == [10, 8, 6]

CF Div 4: 3rd Sept

In [29]:
def minimize(a: int, b: int) -> int:
    return b - a

In [30]:
def legend_of_freya(x: int, y: int, k: int) -> int:
    x_moves, y_moves = math.ceil(x / k), math.ceil(y / k)
    moves = max(x_moves, y_moves) * 2
    return moves if y_moves >= x_moves else moves - 1

# Testing the solution
assert legend_of_freya(9, 11, 3) == 8
assert legend_of_freya(0, 10, 8) == 4
assert legend_of_freya(1000000, 100000, 10) == 199999

In [31]:
def getLucky(s: str, k: int) -> int:
    ords: list[int] = list(map(lambda x: ord(x) - ord('a') + 1, s))
    curr = functools.reduce(lambda x, y: x * 100 + y if y >= 10 else x * 10 + y, ords, 0)
    while k:
        total = 0
        while curr:
            total, curr = total + (curr % 10), curr // 10
        curr, k = total, k - 1
    return curr

# Testing the solution
assert getLucky("leetcode", 2) == 6
assert getLucky("zbax", 2) == 8
assert getLucky("iiii", 1) == 36

LC Practice

In [32]:
def firstBadVersion(N: int) -> int:
    def isBadVersion(version: int) -> bool:
        return True

    low, high = 0, N - 1
    while low <= high:
        mid = (low + high) // 2
        if isBadVersion(mid):
            high = mid - 1
        else:
            low = mid + 1
    return low

LC Practice - 6th Sept 2024

In [33]:
def minOperations(logs: list[str]) -> int:
    depth = 0
    for op in logs:
        if op == "../":
            depth = max(0, depth - 1)
        elif op == "./":
            pass
        else:
            depth += 1
    return depth

# Testing the solution
assert minOperations(["d1/","d2/","../","d21/","./"]) == 2
assert minOperations(["d1/","d2/","./","d3/","../","d31/"]) == 3
assert minOperations(["d1/","../","../","../"]) == 0

In [34]:
def SOE(N: int) -> list[int]:
    soe: list[bool] = [True for i in range(N + 1)]
    soe[0] = soe[1] = False
    i = 2
    while i * i <= N:
        if soe[i]:
            soe[i*i::i] = [False] * len(soe[i*i::i])
        i += 1
    return [i for i in range(N + 1) if soe[i]]

# Testing the solution
assert len(SOE(20)) == 8

In [35]:
def GCD(n1: int, n2: int) -> int:
    while n1 > 0 and n2 > 0:
        n1, n2 = max(n1, n2), min(n1, n2)
        n1, n2 = n2, n1 % n2
    return max(n1, n2)

# Testing the solution
assert GCD(4, 6) == 2
assert GCD(9, 8) == 1
assert GCD(6, 12) == 6

In [36]:
def LCM(n1: int, n2: int) -> int:
    return (n1 * n2) // GCD(n1, n2)

# Testing the solution
assert LCM(4, 6) == 12
assert LCM(3, 5) == 15
assert LCM(4, 12) == 12

In [37]:
def divisors(N: int) -> list[int]:
    divs: list[int] = []
    i = 1
    while i * i <= N:
        if N % i == 0:
            divs.append(i)
            if N // i != i:
                divs.append(N // i)
        i += 1

    return sorted(divs)

# Testing the solution
assert divisors(18) == [1,2,3,6,9,18]

In [38]:
def mostFrequentElement(nums: list[int]) -> int:
    # Count freq of elements
    freq: dict[int, int] = dict()
    for n in nums:
        freq[n] = freq.get(n, 0) + 1

    # Compute max occuring element
    max_freq, max_value = 0, 0
    for v, f in freq.items():
        if f > max_freq:
            max_freq, max_value = f, v
        elif f == max_freq:
            max_value = min(v, max_value)

    return max_value

# Testing the solution
assert mostFrequentElement([2,3,1]) == 1
assert mostFrequentElement([4,4,5,5,6]) == 4
assert mostFrequentElement([1,2,3,3,2,2,2,2]) == 2

In [39]:
def secondMostFrequentElement(nums: list[int]) -> int:
    # Compute the freq of all elements
    freq: collections.Counter[int] = collections.Counter(nums)

    # Return second most freq
    f1, v1, f2, v2 = 0, -1, 0, -1
    for v, f in freq.items():
        if f > f1:
            f1, v1, f2, v2 = f, v, f1, v1
        elif f == f1:
            v1 = min(v1, v)
        elif f > f2:
            f2, v2 = f, v
        elif f == f2:
            v2 = min(v2, v)

    return v2

# Testing the solution
assert secondMostFrequentElement([1,2,2,3,3,3]) == 2
assert secondMostFrequentElement([4,4,5,5,6,7]) == 6
assert secondMostFrequentElement([10,9,7,7]) == 9
assert secondMostFrequentElement([1,2,2,1,3,3]) == -1

In [40]:
def sumHighestAndLowestFrequency(nums: list[int]) -> int:
    # Compute freq
    freq: collections.Counter[int] = collections.Counter(nums)

    # Return max freq + min freq
    max_freq, min_freq = 0, int(1e4)
    for v, f in freq.items():
        max_freq, min_freq = max(max_freq, f), min(min_freq, f)

    return max_freq + min_freq

# Testing the solution
assert sumHighestAndLowestFrequency([4,4,5,5,6]) == 3
assert sumHighestAndLowestFrequency([1,2,2,3,3,3]) == 4
assert sumHighestAndLowestFrequency([1,1]) == 4

In [41]:
def addDigits(num: int) -> int:
    digits = num
    while int(math.log10(num) + 1) > 1:
        curr, digits = num, 0
        while curr:
            digits += curr % 10
            curr //= 10
        num = digits
    return digits

# Testing the solution
assert addDigits(10) == 1
assert addDigits(91) == 1

In [42]:
def checkPrime(num: int) -> bool:
    def backtrack(curr: int) -> bool:
        if num % curr == 0:
            return False
        elif curr * curr > num:
            return True
        else:
            return backtrack(curr + 1)

    return backtrack(2) if num != 1 else False

# Testing the solution
assert checkPrime(41) == True
assert checkPrime(27) == False
assert checkPrime(1) == False

In [43]:
def longestCommonPrefix(strs: list[str]) -> str:
    prefix: list[str] = list(strs[0])
    for st in strs[1:]:
        i = 0
        while i < len(prefix) and i < len(st):
            if prefix[i] != st[i]:
                break
            i += 1
        prefix = prefix[:i]

    return "".join(prefix)

# Testing the solution
assert longestCommonPrefix(["flowers", "flow", "fly", "flight"]) == "fl"
assert longestCommonPrefix(["dog" , "cat" , "animal", "monkey"]) == ""
assert longestCommonPrefix(["lady", "lazy"]) == "la"

In [44]:
def isomorphicString(s1: str, s2: str) -> bool:
    N = len(s1)
    if N != len(s2):
        return False
    else:
        mapping: dict[str, str] = dict()
        rev_mapping: dict[str, str] = dict()
        for i in range(N):
            if s1[i] not in mapping and s2[i] not in rev_mapping:
                mapping[s1[i]], rev_mapping[s2[i]] = s2[i], s1[i]
            elif mapping.get(s1[i], "* MISSING *") != s2[i] or rev_mapping.get(s2[i], "* MISSING *") != s1[i]:
                return False
        return True

# Testing the solution
assert isomorphicString("egg", "add") == True
assert isomorphicString("apple", "bbnbm") == False
assert isomorphicString("paper", "title") == True

In [45]:
def rotateString(s1: str, s2: str) -> bool:
    """
    Logic: Create new string s1 + s2 so that we can search for all
    possible rotations simply by searching for substrings
    """
    N = len(s1)
    if N != len(s2):
        return False
    else:
        double_str = s1 + s1
        for i in range(N):
            j, k = i, 0
            while j < 2 * N and k < N:
                if double_str[j] == s2[k]:
                    j, k = j + 1, k + 1
                else:
                    break
            if k == N:
                return True
        return False

# Testing the solution
for i in range(100):
    length_ = 5
    s1 = random.choices(string.ascii_lowercase, k=length_)
    pivot = random.randint(0, length_ - 1)
    s2 = s1[pivot:] + s1[:pivot]
    result = True
    if random.random() > 0.5:
        s2[random.randint(0, length_ - 1)] = "*"
        result = False
    assert rotateString("".join(s1), "".join(s2)) == result